In [37]:
import optuna
import torch
import torch.nn as nn
from torchvision import datasets
from torch.utils.data import DataLoader,TensorDataset
from torch import optim
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [38]:
x,y=make_classification(n_samples=1000,n_classes=2,random_state=42,n_features=20)

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.25,random_state=42)

In [39]:
x_train_tensor,y_train_tensor=torch.tensor(x_train,dtype=torch.float32),torch.tensor(y_train,dtype=torch.long)
x_test_tensor,y_test_tensor=torch.tensor(x_test,dtype=torch.float32),torch.tensor(y_test,dtype=torch.long)

In [40]:
class SimpleNN(nn.Module):
    def __init__(self,input_dim,hidden_dim):
        super().__init__()
        self.network=nn.Sequential(
            nn.Linear(input_dim,hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim,2)
        )
    def forward(self,x):
        return self.network(x)

In [42]:
def objective(trail):
    hidden_dim=trail.suggest_int('hidden_dim',16,128)
    learning_rate=trail.suggest_loguniform('learning_rate',1e-4,1e-1)
    model=SimpleNN(input_dim=20,hidden_dim=hidden_dim)
    optimizer=optim.Adam(model.parameters(),lr=learning_rate)
    criterion=nn.CrossEntropyLoss()
    epochs=20
    batch_size=64
    train_loader=DataLoader(TensorDataset(x_train_tensor,y_train_tensor),shuffle=True,batch_size=batch_size)
    test_loader=DataLoader(TensorDataset(x_test_tensor,y_test_tensor),shuffle=True,batch_size=batch_size)
    for epoch in range(epochs):
        model.train()
        running_loss=0.0
        for data , labels in train_loader:
            optimizer.zero_grad()
            pred=model(data)
            loss=criterion(pred,labels)
            loss.backward()
            optimizer.step()
            running_loss+=loss.item()

    model.eval()
    y_pred,y_true=[],[]
    with torch.no_grad():
        for data,labels in test_loader:
            pred=model(data)
            loss=criterion(pred,labels)
            _,predicted=torch.max(pred,axis=1)
            y_true.extend(labels.numpy())
            y_pred.extend(predicted.numpy())
    accuracy=accuracy_score(y_true,y_pred)
    
    return accuracy

In [43]:
study=optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=20)

[I 2025-07-16 14:49:35,819] A new study created in memory with name: no-name-baa4b757-ff0c-4a8b-870d-c94d0e00d232
C:\Users\krant\AppData\Local\Temp\ipykernel_20052\241185576.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate=trail.suggest_loguniform('learning_rate',1e-4,1e-1)
[I 2025-07-16 14:49:36,141] Trial 0 finished with value: 0.852 and parameters: {'hidden_dim': 34, 'learning_rate': 0.00232053232319644}. Best is trial 0 with value: 0.852.
C:\Users\krant\AppData\Local\Temp\ipykernel_20052\241185576.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate=trail.suggest_loguniform('learning_rate',1e-4,1e-1)
[I 2025-07-16 14:49:36,376] Trial 1 

In [44]:
study.best_params

{'hidden_dim': 60, 'learning_rate': 0.001530030447326097}